# Imports

In [5]:
import os
from PIL import Image
from tqdm import tqdm

import torch
from torch import nn, optim

import torchvision
from torchvision import models, transforms
from torchvision.utils import save_image


In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Images

In [7]:
def load_image(image_name, filepath='../neural-style-transfer/imgs, device='cuda'):
    image_path = os.path.join(filepath, image_name + ".jpg")
    image = Image.open(image_path).convert('RGB')
    
    transform = transforms.Compose([
        transforms.Resize(size=(224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    image_tensor = transform(image)
    image_tensor = image_tensor.unsqueeze(0).to(device)
    
    return image_tensor

In [34]:
original_img = load_image("katy-perry").to(device)
style_img = load_image("starry-night").to(device)

In [35]:
# generated = torch.randn(original_img.data.shape, device=device, requires_grad=True)
generated = original_img.clone().detach().requires_grad_(True)

# Model

In [36]:
# Our Layers
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.chosen_features = ['0', '5', '10', '19', '28']
        self.model = models.vgg19(pretrained = True).features
    
    def forward(self, x):
        features = []
        for layer_num, layer in enumerate(self.model):
            x = layer(x)
            if str(layer_num) in self.chosen_features:
                features.append(x)
        
        return features

In [37]:
model = VGG().to(device).eval()

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Training

In [38]:
# Hyperparameters
epochs = 6000
learning_rate = 0.001
alpha = 1
beta = 0.01
optimizer = optim.Adam([generated], lr=learning_rate)

for step in tqdm(range(epochs)):
    # Obtain the convolution features in specifically chosen layers
    generated_features = model(generated)
    original_img_features = model(original_img)
    style_features = model(style_img)

    # Loss is 0 initially
    style_loss = original_loss = 0

    # iterate through all the features for the chosen layers
    for gen_feature, orig_feature, style_feature in zip(
        generated_features, original_img_features, style_features
    ):

        # batch_size will just be 1
        batch_size, channel, height, width = gen_feature.shape
        original_loss += torch.mean((gen_feature - orig_feature) ** 2)
        # Compute Gram Matrix of generated
        G = gen_feature.view(channel, height * width).mm(
            gen_feature.view(channel, height * width).t()
        )
        # Compute Gram Matrix of Style
        A = style_feature.view(channel, height * width).mm(
            style_feature.view(channel, height * width).t()
        )
        style_loss += torch.mean((G - A) ** 2)

    total_loss = alpha * original_loss + beta * style_loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if step % 200 == 0:
        print(total_loss)
        save_image(generated, "generated.png")

  0%|          | 5/6000 [00:01<17:58,  5.56it/s]  

tensor(328320.1250, device='cuda:0', grad_fn=<AddBackward0>)


  3%|▎         | 206/6000 [00:09<04:18, 22.40it/s]

tensor(23416.2461, device='cuda:0', grad_fn=<AddBackward0>)


  7%|▋         | 405/6000 [00:18<04:00, 23.22it/s]

...

 93%|█████████▎| 5605/6000 [04:05<00:17, 23.09it/s]

tensor(1146.7263, device='cuda:0', grad_fn=<AddBackward0>)


 97%|█████████▋| 5806/6000 [04:14<00:08, 22.31it/s]

tensor(1074.8008, device='cuda:0', grad_fn=<AddBackward0>)


100%|██████████| 6000/6000 [04:22<00:00, 22.84it/s]
